In [55]:
import pandas as pd
from math import sqrt
import numpy as np

In [81]:
movies_df = pd.read_csv('C:/Users/NISA/Downloads/ml-25m/movies.csv', encoding='latin-1')
ratings_df = pd.read_csv('C:/Users/NISA/Downloads/ml-25m/ratings.csv', encoding='latin-1')
print(movies_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB
None


In [82]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [83]:
movies_df.drop(['genres'], axis = 1, inplace = True)
ratings_df.drop(['timestamp'], axis = 1, inplace = True)

In [84]:
userInput = [{'title':'Harry Potter and the Half-Blood Prince', 'rating':4.7},
             {'title':'Toy Story 3', 'rating':3.8},
             {'title':'Star Wars: Episode VII - The Force Awakens', 'rating':3},
             {'title':'Black Panther', 'rating':5},
             {'title':'Mad Max: Fury Road', 'rating':3.5}]
inputMovies = pd.DataFrame(userInput)
print(inputMovies)

                                        title  rating
0      Harry Potter and the Half-Blood Prince     4.7
1                                 Toy Story 3     3.8
2  Star Wars: Episode VII - The Force Awakens     3.0
3                               Black Panther     5.0
4                          Mad Max: Fury Road     3.5


In [85]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies[['movieId','title','rating']]
print(inputMovies)

   movieId                                       title  rating
0    69844      Harry Potter and the Half-Blood Prince     4.7
1    78499                                 Toy Story 3     3.8
2   122882                          Mad Max: Fury Road     3.5
3   122886  Star Wars: Episode VII - The Force Awakens     3.0
4   122906                               Black Panther     5.0


In [86]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
print(userSubset.head())


     userId  movieId  rating
695       3    69844     3.5
722       3    78499     4.0
859       3   122882     3.5
860       3   122886     3.0
864       3   122906     4.0


In [87]:
userSubsetGroup = userSubset.groupby(['userId'])

def take_5_elem(x):
    return len(x[1])
    

userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:5])


[(3,      userId  movieId  rating
695       3    69844     3.5
722       3    78499     4.0
859       3   122882     3.5
860       3   122886     3.0
864       3   122906     4.0), (647,        userId  movieId  rating
85443     647    69844     3.5
85457     647    78499     3.5
85538     647   122882     4.5
85539     647   122886     1.5
85543     647   122906     3.5), (901,         userId  movieId  rating
125959     901    69844     3.5
125997     901    78499     4.0
126221     901   122882     5.0
126222     901   122886     3.5
126227     901   122906     3.5), (906,         userId  movieId  rating
127536     906    69844     4.5
127546     906    78499     4.0
127637     906   122882     4.0
127638     906   122886     4.0
127642     906   122906     5.0), (997,         userId  movieId  rating
141973     997    69844     2.5
142057     997    78499     4.0
142544     997   122882     3.5
142545     997   122886     4.5
142552     997   122906     5.0)]


In [89]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:

    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')

    nRatings = len(group)

    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]

    tempRatingList = temp_df['rating'].tolist()
   
    tempGroupList = group['rating'].tolist()
   
    
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [90]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

   similarityIndex  userId
0         0.645165       3
1         0.410628     647
2        -0.390996     901
3         0.905246     906
4        -0.093540     997


In [91]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

    similarityIndex  userId
3          0.905246     906
70         0.839506   13617
37         0.661098    6281
6          0.657004    1288
25         0.645165    4688


In [92]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
print(topUsersRating.head(100))

    similarityIndex  userId  movieId  rating
0          0.905246     906        2     4.0
1          0.905246     906       47     4.0
2          0.905246     906       50     5.0
3          0.905246     906      316     4.0
4          0.905246     906      356     4.0
..              ...     ...      ...     ...
95         0.905246     906    68554     3.0
96         0.905246     906    68954     4.0
97         0.905246     906    69406     4.0
98         0.905246     906    69844     4.5
99         0.905246     906    70183     5.0

[100 rows x 4 columns]


In [93]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
print(topUsersRating.head())

   similarityIndex  userId  movieId  rating  weightedRating
0         0.905246     906        2     4.0        3.620982
1         0.905246     906       47     4.0        3.620982
2         0.905246     906       50     5.0        4.526228
3         0.905246     906      316     4.0        3.620982
4         0.905246     906      356     4.0        3.620982


In [94]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

         sum_similarityIndex  sum_weightedRating
movieId                                         
1                   5.433434           21.127836
2                   4.157996           12.312918
3                   0.421068            0.022854
4                   0.602254            1.806762
5                   0.241736            0.557570


In [95]:
recommendation_df = pd.DataFrame()

recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
print(recommendation_df.head())

         weighted average recommendation score  movieId
movieId                                                
1                                     3.888487        1
2                                     2.961262        2
3                                     0.054277        3
4                                     3.000000        4
5                                     2.306524        5


In [102]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
print(recommendation_df.head())

movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

         weighted average recommendation score  movieId
movieId                                                
3258                              2.265549e+13     3258
86142                             1.812439e+13    86142
85796                             1.341343e+03    85796
5247                              5.849545e+01     5247
2572                              3.494221e+01     2572


,movieId,title,year
2481,2572,10 Things I Hate About You,1999
3067,3160,Magnolia,1999
3165,3258,Death Becomes Her,1992
5139,5247,Smokey and the Bandit,1977
5200,5308,Three Men and a Baby,1987
16232,85796,Hobo with a Shotgun,2011
16275,86142,13 Assassins (JÃ»san-nin no shikaku),2010
18247,95558,Beasts of the Southern Wild,2012
19298,100326,Stand Up Guys,2012
54295,189241,Hotel Artemis,2018
